# Questions to be solved
① How do bitrate, network frame rate, and resolution vary over time?
② How do bitrate and network frame-rate vary across stadia, Luna, and GFN?
③ What is the correlation between Bitrate and frame rate across stadia, Luna, and GFN respectively?

### Define some variables in the task
Actually I can't understand well about those tags

In [135]:
time = 'time_ms'
bitrate = 'total_bps'
network_fps = 'net_fps'

# Resolution is defined inside functions, since it have to use width and height in each data matrix

# Q1. Aboute the 3 variables
1. Want to have direct view of the trend over Time.
2. Pick one *parsed_videoReceiveStream.json* file from each platform.
3. Plot the Line Chart of variables vs Time.
4. Observe the 9 plots

In [136]:
# input directories of JSON
files = [
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/gfn/fc5_gfn_lan_lab_0908/parsed_videoReceiveStream.json',
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/luna/fc5_luna_lan_lab_0911/parsed_videoReceiveStream.json',
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/stadia/fc5_stadia_lan_lab_0908/parsed_videoReceiveStream.json'
]

### Define a function:
input:  directories of parsed_videoReceiveStream.json
return: 3 plots (fig1, fig2, fig3)

In [137]:
time = 'time_ms'
bitrate = 'total_bps'
network_fps = 'net_fps'
# input directories of JSON
files = [
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/gfn/fc5_gfn_lan_lab_0908/parsed_videoReceiveStream.json',
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/luna/fc5_luna_lan_lab_0911/parsed_videoReceiveStream.json',
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/stadia/fc5_stadia_lan_lab_0908/parsed_videoReceiveStream.json'
]
import plotly.express as px
import plotly.io as pio
import pandas as pd
import json

def plotQ1(file):
    # Load the file and extract the matrix, name it df
    with open(file, 'r') as f:
        data = json.load(f)

    df_list = []

    for tag, tag_data in data.items():
        count = tag_data['count']
        for i in range(count):
            row_data = {}
            for key, value in tag_data.items():
                if key != 'count':
                    row_data[key] = value[i]
            df_list.append(row_data)

    df = pd.DataFrame(df_list)


    # Convert time_ms to datetime and set as index
    df['time_ms'] = pd.to_datetime(df['time_ms'], unit='ms')
    df.set_index('time_ms', inplace=True)

    # define resolution：if width = 1920, height = 1080, then resolution = '1920*1080'
    df['resolution'] = df['width'].astype(int).astype(str) + '*' + df['height'].astype(int).astype(str)

    # bitrate vs time
    fig1 = px.line(
        df,
        x=df.index,
        y=bitrate,
        title='Bitrate vs Time'
    )

    # nfps vs time
    fig2 = px.line(
        df,
        x=df.index,
        y=network_fps,
        title='Network Frame Rate vs Time'
    )

    # resolution vs time
    fig3 = px.line(
        df,
        x=df.index,
        y='resolution',
        title = 'Resolution vs Time'
    )

    # define 1080p>720p>0, make the plot looks better
    fig3.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': ['0*0', '1280*720', '1920*1080']})
    return fig1, fig2, fig3

### Output the plots into png file

In [138]:
for i in [0,1,2]:
    fig1, fig2, fig3 = plotQ1(files[i])
    fig1.show()
    fig2.show()
    fig3.show()
    # find out which platform is this output
    if i == 0:
        platform = 'gfn'
    if i == 1:
        platform = 'luna'
    if i == 2:
        platform = 'stadia'

    pio.write_image(fig1, './Q1_Output/fig_'+platform+'_bps'+'.png')
    pio.write_image(fig2, './Q1_Output/fig_'+platform+'_nfps'+'.png')
    pio.write_image(fig3, './Q1_Output/fig_'+platform+'_res'+'.png')

# Q2. Comparison of different platforms
1. Want to plot a single Line Chart for each 'Bitrate' and 'NetFPS', 2 plots in total.
2. x-axis is the 3 platforms and y-axis is the value. Each plot compares max, mean, median, min, stdev.
3. First, for each platform, compute the average of the 5 value. Second, plot them into one Line Chart with 5 Lines.

## Define a function:
input: which tag do you want to track
output: Generate a boxplot

In [139]:
import plotly.graph_objects as go
import os

def plotQ2(tag_name):
    # 定义字典来存储每个平台的指标值
    platforms = {'gfn': {}, 'stadia': {}, 'luna': {}}

    # 遍历每个平台文件夹
    for platform in platforms.keys():
        # 获取平台文件夹路径
        folder_path = f'/Users/suihua/Desktop/cloudgaming/task1/Dataset/{platform}'

        # 遍历每个实验文件夹
        for experiment in os.listdir(folder_path):
            experiment_path = os.path.join(folder_path, experiment)

            # 如果实验文件夹不是目录，则跳过
            if not os.path.isdir(experiment_path):
                continue

            # 遍历实验文件夹中的所有JSON文件
            for filename in os.listdir(experiment_path):
                if filename != 'vrs_summary_stats.json':
                    continue

                # 读取JSON文件
                file_path = os.path.join(experiment_path, filename)
                with open(file_path) as f:
                    data = json.load(f)

                # 读取指定tag的值
                value = data.get(tag_name)
                if value is None:
                    continue

                # 更新平台字典中的值
                for key, val in value.items():
                    platform_key = f'{platform}_{tag_name}_{key}'
                    if platform_key not in platforms[platform]:
                        platforms[platform][platform_key] = []
                    platforms[platform][platform_key].append(val)

        # 计算每个平台的指标平均值
        for key in platforms[platform].keys():
            platforms[platform][key] = sum(platforms[platform][key]) / len(platforms[platform][key])

    #  将平台字典转换为数据框
    df = []
    for platform, values in platforms.items():
        for key, val in values.items():
            metric, stat = key.split('_', 1)
            df.append({'Platform': platform.upper(), 'Metric': metric.upper(), 'Stat': stat.upper(), 'Value': val})
    df = pd.DataFrame(df)

    # 使用go.Box绘制盒图
    fig = go.Figure()
    for metric in df['Metric'].unique():
        metric_df = df[df['Metric'] == metric]
        fig.add_trace(go.Box(x=metric_df['Platform'], y=metric_df['Value'], name=metric.upper(), boxpoints=False, boxmean=False))

    # 设置图表标题和轴标签
    fig.update_layout(title=f'{tag_name.upper()} across GFN, Stadia, and Luna', yaxis_title=f'{tag_name.upper()} ')

    # 显示图表
    fig.show()

    # 保存图表
    pio.write_image(fig, './Q2_Output/fig_'+str(tag_name)+'.png')

In [140]:
plotQ2(bitrate)

In [141]:
plotQ2(network_fps)

# Q3. Correlation between Bitrate and Frame Rate
1. Use 1 json file for each platform.
2. draw scatter charts. y-axis: nfps, x-axis: bitrate.

First we read the data:

In [2]:
import plotly.express as px
import plotly.io as pio
import pandas as pd
import json
from scipy.stats import pearsonr

bitrate = 'total_bps'
network_fps = 'net_fps'

files = [
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/gfn/fc5_gfn_lan_lab_0923/parsed_videoReceiveStream.json',
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/luna/fc5_luna_lan_lab_1007/parsed_videoReceiveStream.json',
    '/Users/suihua/Desktop/cloudgaming/task1/Dataset/stadia/fc5_stadia_lan_lab_0911/parsed_videoReceiveStream.json'
]

for i in [0,1,2]:
    file = files[i]
    if i == 0:
        platform = 'gfn'
    if i == 1:
        platform = 'luna'
    if i == 2:
        platform = 'stadia'

    # Load the file and extract the matrix, name it df
    with open(file, 'r') as f:
        data = json.load(f)

    df_list = []

    for tag, tag_data in data.items():
        count = tag_data['count']
        for i in range(count):
            row_data = {}
            for key, value in tag_data.items():
                if key != 'count':
                    row_data[key] = value[i]
            df_list.append(row_data)

    df = pd.DataFrame(df_list)
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31297 entries, 0 to 31296
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   current_delay                       31297 non-null  float64
 1   dec_fps                             31297 non-null  float64
 2   decode_delay                        31297 non-null  float64
 3   first_frame_received_to_decoded_ms  31297 non-null  float64
 4   frame_drop                          31297 non-null  float64
 5   frames_decoded                      31297 non-null  float64
 6   frames_rendered                     31297 non-null  float64
 7   freeze_count                        31297 non-null  float64
 8   height                              31297 non-null  float64
 9   interframe_delay_max_ms             31297 non-null  float64
 10  jb_cuml_delay                       31297 non-null  float64
 11  jb_delay                            31297

In [142]:
    # pearson's correlation coefficient
    x = df[bitrate]
    y = df[network_fps]
    corr, _ = pearsonr(x, y)

    fig = px.scatter(
        df,
        x=bitrate,
        y=network_fps,
        title='Bitrate vs Frame Rate (Corr: {:.2f})'.format(corr),
    )
    pio.write_image(fig, './Q3_Output/fig_'+platform+'.png')
    fig.show()